In [9]:
%pip install -e .
import pandas as pd
import sys
import os
import numpy as np
# Add the erp_app directory to sys.path if 'source' is a local module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from source.components.data_transformation import ColumnTranformer

Obtaining file:///C:/Codex%20Future%20Internship/ERP%20-%20Project/erp_model/erp_app/research
Note: you may need to restart the kernel to use updated packages.


ERROR: file:///C:/Codex%20Future%20Internship/ERP%20-%20Project/erp_model/erp_app/research does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [10]:

data = {
    'id': [101, 102, 103, 104, 105],
    # Mostly dates, but one 'typo' that usually forces the whole col to object
    'join_date': ['2023-01-15', '2023-04-10', 'Unknown', '2023-05-20', '2023-06-01'],
    # Numeric stored as strings with one genuine missing value
    'salary': ['50000', '60000', np.nan, '55000', '70000'],
    # True categorical
    'department': ['Sales', 'Tech', 'Sales', 'HR', 'Tech'],
    # Another date format with a different kind of error
    'last_login': ['01/15/2023', '04/10/2023', '99/99/9999', '05/20/2023', np.nan]
}

df = pd.DataFrame(data)

print("--- Original Data Types ---")
print(df.dtypes)
print("\n--- Original Data ---")
print(df)

--- Original Data Types ---
id             int64
join_date     object
salary        object
department    object
last_login    object
dtype: object

--- Original Data ---
    id   join_date salary department  last_login
0  101  2023-01-15  50000      Sales  01/15/2023
1  102  2023-04-10  60000       Tech  04/10/2023
2  103     Unknown    NaN      Sales  99/99/9999
3  104  2023-05-20  55000         HR  05/20/2023
4  105  2023-06-01  70000       Tech         NaN


In [11]:
transformer = ColumnTranformer()
print("\n--- Starting Auto-Correction ---")
df_corrected = transformer.auto_correct_datatypes(df, threshold=0.25) 
# Note: I used 0.25 threshold here because our mock dataset is tiny (5 rows).
# In a real large dataset, keep threshold low (e.g., 0.01 to 0.05).

print("\n--- Final Data Types ---")
print(df_corrected.dtypes)
print("\n--- Final Data (Cleaned) ---")
print(df_corrected)


--- Starting Auto-Correction ---
✅ Column 'join_date' converted to DATETIME.
✅ Column 'salary' converted to DATETIME.
✅ Column 'department' converted to DATETIME.
✅ Column 'last_login' converted to DATETIME.

--- Final Data Types ---
id                     int64
join_date     datetime64[ns]
salary        datetime64[ns]
department    datetime64[ns]
last_login    datetime64[ns]
dtype: object

--- Final Data (Cleaned) ---
    id  join_date salary department last_login
0  101 2023-01-15    NaT        NaT 2023-01-15
1  102 2023-04-10    NaT        NaT 2023-04-10
2  103        NaT    NaT        NaT        NaT
3  104 2023-05-20    NaT        NaT 2023-05-20
4  105 2023-06-01    NaT        NaT        NaT


c:\Codex Future Internship\ERP - Project\erp_model\source\components\data_transformation.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_corrected = pd.to_datetime(df[col], errors = 'coerce')
c:\Codex Future Internship\ERP - Project\erp_model\source\components\data_transformation.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_corrected = pd.to_datetime(df[col], errors = 'coerce')


In [12]:
df_corrected

,id,join_date,salary,department,last_login
0,101,2023-01-15,NaT,NaT,2023-01-15
1,102,2023-04-10,NaT,NaT,2023-04-10
2,103,NaT,NaT,NaT,NaT
3,104,2023-05-20,NaT,NaT,2023-05-20
4,105,2023-06-01,NaT,NaT,NaT


In [13]:
df

,id,join_date,salary,department,last_login
0,101,2023-01-15,NaT,NaT,2023-01-15
1,102,2023-04-10,NaT,NaT,2023-04-10
2,103,NaT,NaT,NaT,NaT
3,104,2023-05-20,NaT,NaT,2023-05-20
4,105,2023-06-01,NaT,NaT,NaT


In [16]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

N_ROWS = 1000

def generate_messy_data(n=1000):
    # 1. Base Data Generation (Clean-ish start)
    # -----------------------------------------
    # Dates within last 2 years
    base_dates = []
    for x in np.random.randint(0, 730, n):
        days = x
        # If days is a numpy array
        days = days.item()  # Convert numpy.int32 to Python int
        delta = timedelta(days=days)
        base_dates.append(datetime.today() - delta)
    # Numeric values (e.g., salaries or transactions)
    base_numeric = np.random.normal(50000, 15000, n)
    # Categories
    base_cats = np.random.choice(['Sales', 'Marketing', 'Engineering', 'HR', 'Finance'], n)
    
    df = pd.DataFrame({
        'ID': range(1, n + 1),
        'Join_Date_Messy': base_dates,
        'Salary_Messy': base_numeric,
        'Dept_Messy': base_cats,
        'Age_Mixed': np.random.randint(20, 65, n).astype(object) # Start as object to allow mixing
    })

    # 2. Injecting Disruptions
    # ------------------------
    
    # --- A. Disrupting Dates (Join_Date_Messy) ---
    # 15% distinct invalid strings, 5% different formats, 10% NaNs
    mask_str = np.random.rand(n) < 0.15
    mask_fmt = (np.random.rand(n) >= 0.15) & (np.random.rand(n) < 0.20)
    mask_nan = np.random.rand(n) > 0.90
    
    df.loc[mask_str, 'Join_Date_Messy'] = np.random.choice(['TBD', 'Unknown', 'Pending', 'Error 404'], mask_str.sum())
    # Convert some to EU format (DD/MM/YYYY) which confuses default US parsers
    df.loc[mask_fmt, 'Join_Date_Messy'] = df.loc[mask_fmt, 'Join_Date_Messy'].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, datetime) else x)
    df.loc[mask_nan, 'Join_Date_Messy'] = np.nan

    # --- B. Disrupting Numerics (Salary_Messy) ---
    # Convert to object first to allow string injection
    df['Salary_Messy'] = df['Salary_Messy'].astype(str)
    mask_currency = np.random.rand(n) < 0.10
    mask_text = (np.random.rand(n) >= 0.10) & (np.random.rand(n) < 0.15)
    
    # Inject currency symbols causing standard float conversion to fail
    df.loc[mask_currency, 'Salary_Messy'] = '$' + df.loc[mask_currency, 'Salary_Messy'].astype(str).str[:5]
    # Inject pure text
    df.loc[mask_text, 'Salary_Messy'] = np.random.choice(['Not Provided', 'Redacted', '-'], mask_text.sum())

    # --- C. Disrupting Categories (Dept_Messy) ---
    # Inject typos and case inconsistencies
    mask_typo = np.random.rand(n) < 0.10
    mask_case = np.random.rand(n) < 0.20
    
    # Typos
    typo_map = {'Sales': 'Sales_', 'Marketing': 'Mktg', 'Engineering': 'Eng.', 'HR': 'H R', 'Finance': 'Fin'}
    df.loc[mask_typo, 'Dept_Messy'] = df.loc[mask_typo, 'Dept_Messy'].map(typo_map)
    # Case messiness
    df.loc[mask_case, 'Dept_Messy'] = df.loc[mask_case, 'Dept_Messy'].str.lower()

    # --- D. Disrupting Mixed Types (Age_Mixed) ---
    # A column that should be integer but has text mixed in
    mask_age_text = np.random.rand(n) < 0.05
    df.loc[mask_age_text, 'Age_Mixed'] = np.random.choice(['old enough', 'unknown', 'twenty-five'], mask_age_text.sum())

    return df

# Generate the data
df_test = generate_messy_data(N_ROWS)

# Display a comprehensive sample
print(f"--- Generated DataFrame ({N_ROWS} rows) ---")
print(df_test.info())
print("\n--- Sample with Disrupted Values ---")
# Show a sample specifically targeting the messy rows
print(df_test.sample(20).sort_index())

--- Generated DataFrame (1000 rows) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               1000 non-null   int64 
 1   Join_Date_Messy  916 non-null    object
 2   Salary_Messy     1000 non-null   object
 3   Dept_Messy       1000 non-null   object
 4   Age_Mixed        1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
None

--- Sample with Disrupted Values ---
      ID             Join_Date_Messy        Salary_Messy   Dept_Messy  \
25    26                  07/02/2025                   -      Finance   
43    44                         TBD   60811.59573710866          H R   
109  110  2024-01-06 16:59:33.478344   66944.74227458556    Marketing   
158  159  2024-04-12 16:59:33.478344   53408.92113701852  Engineering   
173  174  2025-05-05 16:59:33.478344   55004.65297175896          H R   
190  191

C:\Users\kushw\AppData\Local\Temp\ipykernel_11364\3571780448.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Error 404' 'Unknown' 'TBD' 'Error 404' 'Pending' 'Pending' 'TBD'
 'Error 404' 'TBD' 'TBD' 'Unknown' 'TBD' 'Error 404' 'TBD' 'Error 404'
 'Pending' 'TBD' 'Error 404' 'Pending' 'Error 404' 'Pending' 'Unknown'
 'TBD' 'TBD' 'Pending' 'Unknown' 'Error 404' 'Unknown' 'Error 404'
 'Error 404' 'TBD' 'Pending' 'TBD' 'TBD' 'TBD' 'Error 404' 'TBD'
 'Error 404' 'Pending' 'Error 404' 'Error 404' 'Unknown' 'TBD' 'TBD' 'TBD'
 'TBD' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'TBD' 'Error 404' 'Pending'
 'Unknown' 'Error 404' 'Unknown' 'TBD' 'TBD' 'Error 404' 'Error 404'
 'Error 404' 'TBD' 'TBD' 'Pending' 'Error 404' 'Pending' 'Unknown'
 'Pending' 'Unknown' 'Unknown' 'Pending' 'Error 404' 'Unknown' 'Error 404'
 'TBD' 'Unknown' 'TBD' 'Pending' 'Error 404' 'Pending' 'Unknown' 'TBD'
 'Unknown' 'Pending' 'Error 40

In [18]:
df_corrected = transformer.auto_correct_datatypes(df_test, threshold=0.05)

✅ Column 'Join_Date_Messy' converted to DATETIME.
✅ Column 'Salary_Messy' converted to DATETIME.
✅ Column 'Dept_Messy' converted to DATETIME.
✅ Column 'Age_Mixed' converted to DATETIME.


c:\Codex Future Internship\ERP - Project\erp_model\source\components\data_transformation.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_corrected = pd.to_datetime(df[col], errors = 'coerce')
c:\Codex Future Internship\ERP - Project\erp_model\source\components\data_transformation.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_corrected = pd.to_datetime(df[col], errors = 'coerce')


In [19]:
df_corrected

,ID,Join_Date_Messy,Salary_Messy,Dept_Messy,Age_Mixed
0,1,2025-07-31 16:59:33.478344,NaT,NaT,1970-01-01 00:00:00.000000045
1,2,2024-09-01 16:59:33.478344,NaT,NaT,1970-01-01 00:00:00.000000044
2,3,2025-02-13 16:59:33.478344,NaT,NaT,1970-01-01 00:00:00.000000029
3,4,NaT,NaT,NaT,1970-01-01 00:00:00.000000064
4,5,2025-08-31 16:59:33.478344,NaT,NaT,1970-01-01 00:00:00.000000038
...,...,...,...,...,...
995,996,2024-02-08 16:59:33.482132,NaT,NaT,1970-01-01 00:00:00.000000047
996,997,2024-03-25 16:59:33.482132,NaT,NaT,1970-01-01 00:00:00.000000045
997,998,2024-12-25 16:59:33.482132,NaT,NaT,1970-01-01 00:00:00.000000049
998,999,2024-12-27 16:59:33.482132,NaT,NaT,1970-01-01 00:00:00.000000032
